# Une méthode pour filtrer les données

### Problématique

Les zones d'autorisation sont représentées par des points placés à certains endroits. Ces points sont plus ou moins espacés et semblent remplir l"espace pour former des zones. Les coordonnées des stations météo et des parcs éoliens existants sont également représentées par des points.

Pour filtrer et comparer ces données, il n"est pas possible de simplement soustraire les coordonnées, car la probabilité qu'une station météo se trouve exactement au même endroit qu'un point d'une zone est presque nulle.

Pour résoudre cette problématique, nous avons d'abord essayé la méthode du `ray casting`, qui consiste à projeter une droite dans toutes les directions autour d'un point jusqu'à rencontrer un autre point. Cette méthode est efficace pour les points situés sur le périmètre d'un polygone, mais moins adaptée lorsque les points remplissent l'aire de celui-ci. Nous avons également tenté d'utiliser la méthode `k-means` pour réduire la taille des zones à tester, mais les résultats n'ont pas été concluants.

Nous avons donc décidé de repartir de zéro en reformulant la question : **À quelle distance d'une zone se trouve une station météo ?**

Cette approche introduit immédiatement la notion de rayon autour d'un point et élimine la nécessité de superposer les points.

Après quelques recherches, nous avons découvert la formule de `Haversine`. Il s'agit d'une formule utilisée pour calculer la distance entre deux points sur une sphère à partir de leurs coordonnées de longitude et de latitude. En termes plus simples, elle permet de calculer la distance la plus courte entre deux points à la surface d'un objet sphérique, comme la Terre. Il suffisait ensuite de préciser une distance maximale autorisée, et le problème était résolu.

In [5]:
import random

import pandas as pd
import numpy as np
import plotly.express as px

In [6]:
R = 6371  # Earth Radius in kilometers
MAX_DIST = 30 # kilometers

In [7]:
df_zones = pd.read_csv("s3://jedha-final-project-jrat/zones.csv")
df_zones.head()

,Y,X,Latitude,Longitude,Color
0,79,571,50.198832,3.355932,"rgb(206,136,255)"
1,80,571,50.188113,3.355932,"rgb(206,136,255)"
2,80,579,50.188113,3.480975,"rgb(208,142,255)"
3,81,572,50.177394,3.371562,"rgb(206,136,255)"
4,83,577,50.155957,3.449714,"rgb(208,142,255)"


In [8]:
df_positions = pd.DataFrame({
    "Latitude": [random.uniform(43, 50) for _ in range(200)],
    "Longitude": [random.uniform(-1, 10) for _ in range(200)]
})
df_positions.head()  # remplacer par les vraies positions des stations meteo ou parcs eoliens

,Latitude,Longitude
0,48.132028,6.578735
1,44.308478,1.334661
2,49.255882,4.396361
3,46.773663,9.434161
4,46.117363,9.247404


In [9]:
def haversine(lat1: float, lon1: float, lat2: float, lon2: float) -> float:
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])

    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))

    return c * R


def check_authorization(df_zones: pd.DataFrame, df_positions: pd.DataFrame):
    authorized_positions = []

    for _, pos in df_positions.iterrows():
        pos_lat, pos_lon = pos["Latitude"], pos["Longitude"]
        is_authorized = False
        
        for _, zone in df_zones.iterrows():
            zone_lat, zone_lon, zone_radius = zone["Latitude"], zone["Longitude"], zone['Radius']
            distance = haversine(pos_lat, pos_lon, zone_lat, zone_lon)
            
            if distance <= zone_radius:
                is_authorized = True
                break

        authorized_positions.append(is_authorized)

    df_positions["is_authorized"] = authorized_positions
    return df_positions


df_zones["Radius"] = MAX_DIST
df_positions_filtered = check_authorization(df_zones, df_positions)

df_positions_filtered.head()


,Latitude,Longitude,is_authorized
0,48.132028,6.578735,True
1,44.308478,1.334661,False
2,49.255882,4.396361,True
3,46.773663,9.434161,False
4,46.117363,9.247404,False


In [10]:
df_positions["is_authorized"] = df_positions["is_authorized"].astype(str)

# zones
fig = px.scatter_mapbox(
    df_zones, 
    lat="Latitude", 
    lon="Longitude",
    zoom=5,
    height=800
)

fig.update_traces(marker=dict(size=MAX_DIST, color="gray"))

# stations or parks
fig.add_scattermapbox(
    lat=df_positions["Latitude"], 
    lon=df_positions["Longitude"], 
    mode="markers",
    marker=dict(size=10),
    marker_color=df_positions["is_authorized"].map({"True": "green", "False": "red"})
)

fig.update_layout(mapbox_style="open-street-map")
fig.show()


![](https://raw.githubusercontent.com/tristanGIANDO/jedha_final_project/develop/jedha_final_project/src/filter_haversine.png)